In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


In [3]:
date: str = "2022年8月12日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

mg_input_list_problem_size: list[int] = [32, 64, 128, 256]
mg_input_list_nit: list[int] = [4, 10, 20, 35]

mg_target_list_problem_size: list[int] = [512]
mg_target_list_nit: list[int] = [50]

In [4]:
from statistics import LinearRegression


class Model_squareRootOfProcess_ForMultipleRegression(ModelBaseForMultipleRegression):
    """プロセス数を表す説明変数に平方根をかけたモデル

    Y = a * X ** 1/2 + b

    Attributes:
        explanatoryVariableColumnNames (list[str]): 説明変数の列名のリスト
        rawExplanatoryVariable (pd.DataFrame): 説明変数のデータフレーム
        rawExplanatoryVariableForTest (pd.DataFrame): テスト用の説明変数のデータフレーム。説明変数のデータフレームと同様の値が入っている(?)
        rawResponseVariable (pd.DataFrame): 目的変数のデータフレーム
        rawResponseVariableForTest (pd.DataFrame): テスト用の目的変数のデータフレーム。目的変数のデータフレームと同様の値が入っている(?)
        responseVariableColumnNames (list[str]): 目的変数の列名のリスト

    """

    def build_model(self) -> bool:
        """build_model(self) -> bool
        
        オブジェクトの初期化時に生成された、インスタンスの説明変数およびインスタンスの目的変数からモデルを構築する
        """

        df_mid_var :pd.DataFrame = self.process_df(
            inputDF = self.rawExplanaoryVariable
        )
        self.lr = LinearRegression()
        self.lr.fit(df_mid_var, self.rawResponseVariable)

        return True

    def process_df(self, inputDF:pd.DataFrame) -> pd.DataFrame:
        """process_df(self, inputDF:pd.DataFrame) -> pd.DataFrame

        inputDF内にあるprocess列を1/2乗した列を追加する
        
        Args:
            inputDF (pd.DataFrame) : 入力DF

        """

        returnDF :pd.DataFrame = inputDF.copy(deep=True)
        returnDF["square_root_of_process"] = np.sqrt( returnDF["process"] )
        reutrnDF = returnDF.drop("process", axis=1)

        return returnDF


def test_Model_squareRootOfProcess_ForMultipleRegression():
    """test_Model_squareRootOfProcess_ForMultipleRegression()

    クラスModel_squareRootOfProcess_ForMultipleRegressionのテスト
    """

    # 説明変数
    plot_process: np.ndarray = np.linspace(10, 20, 11)
    plot_other: np.ndarray = -1 * plot_process
    plot_other2 : np.ndarray = 5 * plot_process

    # 切片・係数
    a :int = 8
    b : int = -37

    # 目的変数
    plot_call: np.ndarray = np.sqrt(plot_process)

    # DFの作成
    columnNames: list[str] = ["process", "plot_other", "plot_other2", "plot_call"]
    datumForDF: list[np.ndarray] = [plot_process, plot_other, plot_other2, plot_call]
    inputDFForTest :pd.DataFrame = pd.DataFrame(index=columnNames, data=datumForDF).T
    inputDFForTest["functionName"] = "functionName"

    # 説明変数のカラム名のリスト
    columnNamesForExp :list[str] = ["process", "plot_other", "plot_other2"]
    # 目的変数のカラム名のリスト
    columnNamesForRes :list[str] = ["plot_call"]

    # 予測の実施
    objectModel = Model_squareRootOfProcess_ForMultipleRegression(
        inputDF = inputDFForTest,
        explanatoryVariableColumnNames = columnNamesForExp,
        responseVariableColumnNames = columnNamesForRes,
        conditionDictForTest={},
    )
    
    # モデルの構築
    objectModel.build_model()
    # モデル構築に用いたデータと予測されたデータとのMAPEを比較して、実装ができているかを確認
    mape :float = objectModel.returnMAPE()
    assert 0 <= mape < 1, f"mape = {mape}"

In [5]:
test_Model_squareRootOfProcess_ForMultipleRegression()

TypeError: LinearRegression.__new__() missing 2 required positional arguments: 'slope' and 'intercept'

In [ ]:
%reset

In [ ]:
input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

In [ ]:
cg_list_exp: list[str] = ["process", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

In [ ]:
# 精度の低い関数を抽出したうえでチューニングする

targetFunctionNames: list[str] = [
    "VECSET",
    "MPI_Wait()",
    "MPI_Send()",
    "SPRNVC",
    "RANDLC",
    "MPI_Irecv()",
    "ICNVRT",
]

In [ ]:
result_series_list: list[pd.DataFrame] = []

function_names: list[str] = list(set(input_rawDF_cg["Name"].tolist()))

input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

In [ ]:
resultDF

In [ ]:
df_ICNVRT = input_rawDF_cg[input_rawDF_cg["functionName"] == "ICNVRT"]
df_Wait = input_rawDF_cg[input_rawDF_cg["functionName"] == "MPI_Wait()"]
df_Send = input_rawDF_cg[input_rawDF_cg["functionName"] == "MPI_Send()"]
df_RANDLC = input_rawDF_cg[input_rawDF_cg["functionName"] == "RANDLC"]
df_VECSET = input_rawDF_cg[input_rawDF_cg["functionName"] == "VECSET"]
df_Irecv = input_rawDF_cg[input_rawDF_cg["functionName"] == "MPI_Irecv()"]
df_SPRNVC = input_rawDF_cg[input_rawDF_cg["functionName"] == "SPRNVC"]

In [ ]:
input_rawDF_cg.columns.tolist()

In [ ]:
target_rawDF_cg.sort_values("#Call", ascending=False)

In [ ]:
print(
    target_rawDF_cg.sort_values("#Call", ascending=False)[:6]
    .loc[:, ["functionName", "#Call"]]
    .style.to_latex(
        caption="ベンチマークプログラムCGで関数コール回数の多い関数（プロセス数256, 初期変数na＝15000, 初期変数nonzer＝21, 初期変数niter＝100, 初期変数shift＝200）",
        label=f"{date}_func_table",
        position_float="centering",
        siunitx=True
    )
)

In [ ]:
# ICNVRT

fig = px.scatter_3d(df_ICNVRT, x="process", y="na", z="#Call")
fig.show()

***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___